[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/latent-consistency-model-colab/blob/main/wip/lcm_sdxl_controlnet_ip_adapter_plus_draw_colab.ipynb)

In [ ]:
!pip install git+https://github.com/huggingface/diffusers -U
!pip install -q controlnet-aux transformers accelerate peft gradio==3.50.2
!pip install -q https://download.pytorch.org/whl/cu121/xformers-0.0.22.post7-cp310-cp310-manylinux2014_x86_64.whl
!pip install -q git+https://github.com/tencent-ailab/IP-Adapter einops

!mkdir /content/models_xl
!wget https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter-plus_sdxl_vit-h.safetensors?download=true -O /content/models_xl/ip-adapter-plus_sdxl_vit-h.safetensors
!wget https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter_sdxl.safetensors?download=true -O /content/models_xl/ip-adapter_sdxl.safetensors

!mkdir /content/image_encoder_xl
!wget https://huggingface.co/h94/IP-Adapter/raw/main/sdxl_models/image_encoder/config.json -O /content/image_encoder_xl/config.json
!wget https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/image_encoder/model.safetensors?download=true -O /content/image_encoder_xl/model.safetensors

import torch
from diffusers import StableDiffusionXLPipeline, UNet2DConditionModel, LCMScheduler
from PIL import Image
from ip_adapter import IPAdapterXL

unet = UNet2DConditionModel.from_pretrained("latent-consistency/lcm-sdxl", torch_dtype=torch.float16, variant="fp16")
pipe = StableDiffusionXLPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", unet=unet, torch_dtype=torch.float16, variant="fp16")
pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config)
ip_model = IPAdapterXL(pipe, "/content/image_encoder_xl", "/content/models_xl/ip-adapter_sdxl.safetensors", "cuda")

In [ ]:
from diffusers.utils import load_image, make_image_grid
image = load_image("https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7WA/fab28432-7d00-43c4-fbd6-ac5989ebf000/width=512/326152.jpeg")
image.resize((512, 768))

images = ip_model.generate(width=512, height=768, pil_image=image, num_samples=1, num_inference_steps=4, guidance_scale=1.0, prompt="girl wearing a red dress", scale=0.5)
make_image_grid(images, rows=1, cols=1)

In [ ]:
image = pipe("one girl wearing a red dress", num_inference_steps=4, guidance_scale=8.0).images[0]
image